In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', '{:.3f}'.format)

In [8]:
OUTPUT_DIR="../../output/openmpi_dinf/experimental"

EXECUTIONS = 50
CITIES = [13, 14, 15]
PROCESSES = [1, 2, 3, 4, 5, 10]
ITERATIONS = 10

In [9]:
def geometric_mean_of_mean_of_instances(mean_of_instances):
    geometric_mean = mean_of_instances.prod()**(1.0/EXECUTIONS)
    return geometric_mean

def std_mean_of_mean_of_instances(mean_of_instances):
    # std_mean = np.std(mean_of_instances)/(EXECUTIONS**2)
    std_mean = np.std(mean_of_instances)
    return std_mean

def extract_time_stats_from_file(file_name):
    file = open(file_name)
    data = []
    for exec in range(1, EXECUTIONS + 1):
        data.append(float(file.readline().split(" ")[1]))
    np_array = np.array(data)
    file.close()
    return np.sum(np_array), np_array


In [10]:
df_time = pd.DataFrame(columns = ["Version", "Cities", "Processes", "Instance_mean", "Instance_std", "Instance_std_%(x100)", "Sum_mean", "Sum_std", "Sum_std_%(x100)"])

for city in list(CITIES):
    sum_array, array_of_instance_array = [], []
    for iteration in range(1,ITERATIONS + 1):
        file_name = OUTPUT_DIR + "/" + str(iteration) + "_seq_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
        
        sum, instance_array = extract_time_stats_from_file(file_name)
        
        array_of_instance_array.append(instance_array)
        sum_array.append(sum)
    
    mean_of_instances = np.mean(array_of_instance_array, axis=0)
    
    geometric_mean = geometric_mean_of_mean_of_instances(mean_of_instances)
    std_mean = std_mean_of_mean_of_instances(mean_of_instances)
    list_row = ["Sequential", city, "1", geometric_mean, std_mean, std_mean/geometric_mean*100, np.mean(sum_array), np.std(sum_array), np.std(sum_array)/np.mean(sum_array)*100]
    df_time.loc[len(df_time)] = list_row
            
    for process in list(PROCESSES):
        sum_array, array_of_instance_array = [], []
        for iteration in range(1,ITERATIONS + 1):
            file_name = OUTPUT_DIR + "/" + str(iteration) + "_par_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_" + str(process) + "_processes_result.out"
            
            sum, instance_array = extract_time_stats_from_file(file_name)

            array_of_instance_array.append(instance_array)
            sum_array.append(sum)

        mean_of_instances = np.mean(array_of_instance_array, axis=0)
        
        geometric_mean = geometric_mean_of_mean_of_instances(mean_of_instances)
        std_mean = std_mean_of_mean_of_instances(mean_of_instances)
        list_row = ["Parallel", city, process, geometric_mean, std_mean, std_mean/geometric_mean*100, np.mean(sum_array), np.std(sum_array), np.std(sum_array)/np.mean(sum_array)*100]
        df_time.loc[len(df_time)] = list_row
df_time

,Version,Cities,Processes,Instance_mean,Instance_std,Instance_std_%(x100),Sum_mean,Sum_std,Sum_std_%(x100)
0,Sequential,13,1,0.463,0.475,102.551,29.897,0.064,0.215
1,Parallel,13,1,0.504,0.459,90.949,31.323,0.040,0.128
2,Parallel,13,2,0.259,0.230,88.899,15.948,0.021,0.135
3,Parallel,13,3,0.178,0.157,88.393,10.953,0.031,0.283
4,Parallel,13,4,0.138,0.121,87.192,8.471,0.023,0.276
5,Parallel,13,5,0.113,0.098,86.555,6.918,0.015,0.220
6,Parallel,13,10,0.069,0.056,80.993,4.142,0.016,0.384
7,Sequential,14,1,1.864,4.691,251.680,156.585,0.112,0.071
8,Parallel,14,1,1.916,4.514,235.667,155.417,0.118,0.076
9,Parallel,14,2,0.977,2.257,230.939,78.626,0.025,0.032


Speedup and Efficiency

In [11]:
df_speedup_efficiency = pd.DataFrame(columns = ["Cities", "Processes", "Instance_speedup", "Instance_efficiency", "Sum_speedup", "Sum_efficiency"])
# df_speedup_efficiency = pd.DataFrame(columns = ["Cities", "Processes", "Sum Speedup", "Sum Efficiency"])

df_sequential = df_time[df_time['Version'].isin(["Sequential"])]
df_parallel = df_time[df_time['Version'].isin(["Parallel"])]

for city in list(CITIES):
    df_seq = df_sequential[df_sequential["Cities"].isin([city])]
    for process in list(PROCESSES):
        df_par = df_parallel[df_parallel["Cities"].isin([city]) & df_parallel["Processes"].isin([process])]
        speedup_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()
        efficiency_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()/float(process)
        speedup_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()
        efficiency_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()/float(process)

        list_row = [city, process, speedup_instance, efficiency_instance, speedup_sum, efficiency_sum]
        # list_row = [city, process, speedup_sum, efficiency_sum]
        df_speedup_efficiency.loc[len(df_speedup_efficiency)] = list_row

df_speedup_efficiency["Processes"] = df_speedup_efficiency["Processes"].astype(int)
df_speedup_efficiency["Cities"] = df_speedup_efficiency["Cities"].astype(int)
df_speedup_efficiency

,Cities,Processes,Instance_speedup,Instance_efficiency,Sum_speedup,Sum_efficiency
0,13,1,0.917,0.917,0.954,0.954
1,13,2,1.788,0.894,1.875,0.937
2,13,3,2.599,0.866,2.730,0.910
3,13,4,3.344,0.836,3.529,0.882
4,13,5,4.085,0.817,4.321,0.864
5,13,10,6.695,0.669,7.219,0.722
6,14,1,0.973,0.973,1.008,1.008
7,14,2,1.907,0.954,1.992,0.996
8,14,3,2.731,0.910,2.862,0.954
9,14,4,3.545,0.886,3.763,0.941


Output 1

In [12]:
# translate to portuguese
df_time.loc[df_time["Version"] == "Parallel", "Version"] = "Paralelo"
df_time.loc[df_time["Version"] == "Sequential", "Version"] = "Sequencial"
res = df_time.rename(columns={
    "Version": "Versão",
    "Processes": "Processos",
    "Cities": "# Cidades",
    "Instance_mean": "Média geométrica instância",
    "Instance_std": "Desvio padrão instância",
    "Instance_std_%(x100)": "% Desvio padrão instância (x100)",
    "Sum_mean": "Média aritmética tempo total",
    "Sum_std": "Desvio padrão tempo total",
    "Sum_std_%(x100)": "% Desvio padrão tempo total (x100)"
})
res

,Versão,# Cidades,Processos,Média geométrica instância,Desvio padrão instância,% Desvio padrão instância (x100),Média aritmética tempo total,Desvio padrão tempo total,% Desvio padrão tempo total (x100)
0,Sequencial,13,1,0.463,0.475,102.551,29.897,0.064,0.215
1,Paralelo,13,1,0.504,0.459,90.949,31.323,0.040,0.128
2,Paralelo,13,2,0.259,0.230,88.899,15.948,0.021,0.135
3,Paralelo,13,3,0.178,0.157,88.393,10.953,0.031,0.283
4,Paralelo,13,4,0.138,0.121,87.192,8.471,0.023,0.276
5,Paralelo,13,5,0.113,0.098,86.555,6.918,0.015,0.220
6,Paralelo,13,10,0.069,0.056,80.993,4.142,0.016,0.384
7,Sequencial,14,1,1.864,4.691,251.680,156.585,0.112,0.071
8,Paralelo,14,1,1.916,4.514,235.667,155.417,0.118,0.076
9,Paralelo,14,2,0.977,2.257,230.939,78.626,0.025,0.032


In [13]:
res1 = res.pivot(index=["Processos", "Versão"], columns="# Cidades", values=["Média geométrica instância", "Desvio padrão instância", "% Desvio padrão instância (x100)"])
res2 = res.pivot(index=["Processos", "Versão"], columns="# Cidades", values=["Média aritmética tempo total", "Desvio padrão tempo total", "% Desvio padrão tempo total (x100)"])
res1 = res1.sort_values(by='Versão', ascending=False)
res2 = res2.sort_values(by='Versão', ascending=False)
res1

Média geométrica instância              \
# Cidades                                    13    14    15   
Processos Versão                                              
1         Sequencial                      0.463 1.864 9.283   
          Paralelo                        0.504 1.916 9.224   
2         Paralelo                        0.259 0.977 4.741   
3         Paralelo                        0.178 0.683 3.291   
4         Paralelo                        0.138 0.526 2.586   
5         Paralelo                        0.113 0.434 2.135   
10        Paralelo                        0.069 0.264 1.337   

                     Desvio padrão instância               \
# Cidades                                 13    14     15   
Processos Versão                                            
1         Sequencial                   0.475 4.691 11.407   
          Paralelo                     0.459 4.514 11.028   
2         Paralelo                     0.230 2.257  5.584   
3         Paralelo                     0.157 1.578  3.806   
4         Paralelo                     0.121 1.163  2.932   
5         Paralelo                     0.098 0.942  2.384   
10        Paralelo                     0.056 0.514  1.383   

                     % Desvio padrão instância (x100)                  
# Cidades                                          13      14      15  
Processos Versão                                                       
1         Sequencial                          102.551 251.680 122.881  
          Paralelo                             90.949 235.667 119.563  
2         Paralelo                             88.899 230.939 117.773  
3         Paralelo                             88.393 231.208 115.667  
4         Paralelo                             87.192 221.257 113.372  
5         Paralelo                             86.555 217.059 111.656  
10        Paralelo                             80.993 194.759 103.444

In [14]:
res1 = res1.T.swaplevel(1)
res1 = res1.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)
res2 = res2.T.swaplevel(1)
res2 = res2.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)

In [15]:
res1

In [16]:
res2

Output 2

In [17]:
# translate to portuguese
res = df_speedup_efficiency.rename(columns={"Cities": "# Cidades", "Processes": "Processos", "Instance_speedup": "Speedup instância", "Instance_efficiency": "Eficiência instância", "Sum_speedup": "Speedup tempo total", "Sum_efficiency": "Eficiência tempo total"})
res

,# Cidades,Processos,Speedup instância,Eficiência instância,Speedup tempo total,Eficiência tempo total
0,13,1,0.917,0.917,0.954,0.954
1,13,2,1.788,0.894,1.875,0.937
2,13,3,2.599,0.866,2.730,0.910
3,13,4,3.344,0.836,3.529,0.882
4,13,5,4.085,0.817,4.321,0.864
5,13,10,6.695,0.669,7.219,0.722
6,14,1,0.973,0.973,1.008,1.008
7,14,2,1.907,0.954,1.992,0.996
8,14,3,2.731,0.910,2.862,0.954
9,14,4,3.545,0.886,3.763,0.941


In [18]:
res1 = res.pivot(index="Processos", columns="# Cidades", values=["Speedup instância", "Eficiência instância"])
res2 = res.pivot(index="Processos", columns="# Cidades", values=["Speedup tempo total", "Eficiência tempo total"])
res1

Speedup instância             Eficiência instância            
# Cidades                13    14    15                   13    14    15
Processos                                                               
1                     0.917 0.973 1.006                0.917 0.973 1.006
2                     1.788 1.907 1.958                0.894 0.954 0.979
3                     2.599 2.731 2.821                0.866 0.910 0.940
4                     3.344 3.545 3.589                0.836 0.886 0.897
5                     4.085 4.297 4.347                0.817 0.859 0.869
10                    6.695 7.061 6.945                0.669 0.706 0.695

In [19]:
res1 = res1.T.swaplevel(1)
res1 = res1.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)
res2 = res2.T.swaplevel(1)
res2 = res2.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)

In [20]:
res1

In [21]:
res2